In [51]:
!pip install openai
import openai
import base64
from bs4 import BeautifulSoup
import requests

In [56]:
def getPageAsText(url):
  # Fetch the webpage
  response = requests.get(url)
  html = response.text
  soup = BeautifulSoup(html, 'html.parser')

  # Find all image tags
  images = soup.find_all('img')

  # Find all a tags
  pageLinks = soup.find_all('a')

  for img in images:
      img_url = img.get('src')
      # Ignore the footer image that appears on every page
      if img_url == '/umiacs/resources/assets/poweredby_mediawiki_88x31.png':
        continue

      # If the image URL is relative, we need to join it with the base URL
      if img_url.startswith('/'):
          img_url = requests.compat.urljoin(url, img_url)

      try:
          # Download the image
          img_data = requests.get(img_url).content

          # Convert the image to Base64
          img_base64 = base64.b64encode(img_data).decode('utf-8')

          # Create the base64 image string in the format: 'data:image/{ext};base64,{data}'
          # Each image is surrounded by 2 unicode characters to make it find the start/end of an image given the text
          img_format = img_url.split('.')[-1]  # Get the file extension
          base64_src = f'\u3016 data:image/{img_format};base64,{img_base64} \u3017'

          # Replace the src attribute with the base64 string
          p_tag = soup.new_tag('p')
          p_tag.string = base64_src

          # Replace the <img> tag with the new <p> tag
          img.replace_with(p_tag)

      except Exception as e:
          print(f"Error processing image {img_url}: {e}")

  for link in pageLinks:
    href = link.get('href')
    prefixes = ['/umiacs/index.php', '#', 'https://wiki.umiacs.umd.edu/umiacs/index.php', 'javascript']
    if href and not href.lower().startswith(tuple(prefixes)):
      p_tag = soup.new_tag('p')
      p_tag.string = link.getText() + f"(The prior word links to {href})"
      link.replace_with(p_tag)



  # Get the modified HTML as a string
  modified_html = soup.find('body').get_text()

  # Return all the text surrounded by two unicode characters to make it easier to find where each page starts/ends
  return "\u3018" + url + " " + modified_html + "\u3019"

In [ ]:
import requests
def parseEncodedImage(encodedImage):
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {openai.api_key}"
    }

    payload = {
      "model": "gpt-4o-mini",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": "What’s in this image?"
            },
            {
              "type": "image_url",
              "image_url": {
                "url": encodedImage
              }
            }
          ]
        }
      ],
      "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    return response.json()
parseEncodedImage('''data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAiAAAAHICAIAAAD4FMY5AAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAADneSURBVHhe7d19dFTnfeDx8doVkEVujOpRIe0RqS3tdq1JWoxl6awDfxhxjmGcVGxj+SQYb5h4T41bI3zWBjmWT2tRC5yDkDe1c7b20NhxsyhNUDYekRaUbiFtJCAop5kJ2wpnjRobosGidYQMUu1mn7d7587MndHo5WFG8P0cbO595t7n3ufeZ+5vnpcZrhsfHw8AADDXUgFmYmJidHT04sWL77//vk4BAGDGTIAR0eWnP/3pRz/60YULF/7SL/2Sfg0AgBmTAUZHl4997GMmDQCAWft34r933nlHtF30OgAAc0IGGNGIWbhwoV4HAGBOyADz/vvvM+4CAJhbMsAAADDnCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACuuGx8fP3Xq1MqVK03ClTV6aOu6tgGxsHTLV7/1YI1OzDJ25Kk1TxzWy/VtB58PV+jlLInOT0S6J8VCU9fRHQ0LdOI0jca2rmuXp9QcPfZYrU6D19hQbO/g8qfvt3px3NuQ1+Kq6qVVtRs23N/YsLzcpMHPNG5aovPOSLdZLlzeN+aVVRpvYU8NvoYfJUVuwVTUrK5WC+cGhs6qBR8TiX4TXYSBgcSYWcxyZmhQRpdAoLGhdmbRBflNnDn0zMZ1D7T35rxZV9bF4dOnj/bsbmlec/fjsRI5p5JTajcN15Bid5Etr21cqhYGE8MTaiHb6f5e+VdZWZn863B/IseGo0NHTquF+vpaPs5acTHR23taB/Erpbq5/UV/XW3bmhtvW6w3u3i0fUvnyVxV6Jo23Zu25PZNfhpvM6/f1mhS0qxert6egFfRx2Bqbl+lK2bvoI4OWYZOHpXvjqVNTevVem+//4YTQ4O6QVq9urY0muqYAxU1t+fQEL7/sZ37vtvTvr5KbXmuu33/kFrCbCxb/Ygft3et9n6TkmYDH+qQrfiD/FW3r1Z/T/YPnVELGYb6e86Jv8pWrX2ooVEmTB496fsYGR7S8WVpY+1ytYBrwrK1259u1s3gcz39RBigdBQ/wJTX1terhdODwz6jK2cSh2V8CaxvqHa2PHc44ROKnA3LVt2ea7IArk4Lalev0kvnBodG9RJQXIsr9Keea1vxA0ygotYM9B85Oaz+9hpN6HEVNWzvbHn6SCLrMTI2bLrYRCRSf6cbTcQ6H9943913Kp+4Z+Pjuw70n8k5XSC3ibNpOd1938NPvRDLPh3X2Jn+A7u82+c58mhsq9qoMyHXxoYOiQPd8wmZIk+4M+0woyf3P/WwyVWexP6T+Z6s4izE5iazAs46W6JT7uhO7OqOqIzu3BrLyGQ65Z07U03pmFH5dZ3x3IBDQ6oY+lKkF92kZV8PY6rXp1s/ZTV8wa0Aua5zoTdtzjg1WB5h4kxs12Z19VT97D/rHSAbO3NSnr97R2SRVRVP2ypL+n2cwdv4bGyrOeJ9u/qzr8I06olzR+WbdfTkK4+rW6Hfp7qaLChnUKokAow70D85mNVJNpYYUG8OM2zvbOkzlWzopJ5plj2BbCzxyuZPrIu0dx89PXxRJ01e0HOP1m185sg0ptacPfTMxjVNaTldHB48/Gp7ZN19fhnJ7dc1t+zu8W7vHvlQ7iNPjPZ3blz3QJs40AU1OitPuLs98jtb1VSps0eeuW/dlr2HB02u8iT2bll3X6fv9AdxFvetaW4Rm5vMUmd998P7h+ZwXHzG5Z2tM0ODeqGsannW8NtMyj9x5sDjd+s647kBbQ+s2/zKXF4wZdr1c2Lolc2yGr7qVoDUdV7jWxGL4EzsiQfae36srp683PuHLjrvS11Ntsjzd++ILLKq4k1rclziscT+h+9Ov4/uZSrwrojo0tw+IPetaur68o6G9Koy0/fJWKLz4S0vHlW3QlWTI85TbIGJMEsrrt1QUwoBJlBTq3s4sjrJnAnK7rC9MyUgayrZUOKo+jtzAtlEojMSefHHssKULalv2q5mJHW1bdKzjyZP9z7RrB/aU5K1s03Pxknl1L7NTGMa7n2iJf357r/99qb6JbIE4shtOY88sPvhlu7Tk4tva9zU1qWPUq2KPTnQ/tQrBzpbnugdLqtOvWhmUg13P7w3cx6VPgvVMnSzS53FxcG9DzzgH5YyVW1Qezab5uGqbXL1xRcfWmFmcc2mvLN0NrZ3r269Lm1am9F8nVH5z4qH4+6j6tmdcZknf/xipH2/6oqdG9Ovn2NH2s0OokDNegdRomZTRWRFfMUZiJryptkyuLtFPMkXr2iW105e7vrPN+qO69FDTjWR5d2mb4g6f1OLxSV+4oXMW6Iu095BdUvc+9jV1mwql7grT0xVt2RNyBNdZvw+6Xni4W75btwm7oS6cc33r9aPoKVVK9TfgaqKa3jS0fj4+IkTJ35RXD//my/UKXt+YFK0+J67ZOqnvvKPJuEXl7/foTa8a0/cpCjvvP6oSv7sN980KYrIV2VQd9fnvhL/uUl0vPP9jk+rF+seff0dkyY4WdWlHcHNqe7THd/3bC38/Ad7TD6e8/zHr5g0nyP/PP6VzzmZpfYQ3EOL3R795puXTbJ0Oe4cRPr0nh+k5fn264+aDNMvoHsWn/6jv3rbpDl+/o/fNDt9ak/ce6h8/C+OMKPy5uMeKe3mpLn883fe/ME3/+izJu+6u77wNxlHnlH5Pbc64zJfftPZRUo7sfgen0QP/9dnUD/f/ubnVNqn9vwg86a5teCujrTXct606XGKkC8bbw3u+H5micQV/EGHOcVHX8+8Ibmrsad2ZbwtvPv8979yjpdd4Lf/5o+cmpD59pVmUk/cy1FX97n/lfbUcRRQha9+JdGCSQ30H014ZwGZCcreYfsFtX5TyZyetLIVNd4JZEMH9h5WGTQ99+UHs2ZRVjTseG67mts/sPsVNeqRm5NToL6tK/PDT/ntj+3conruzr0c0/mMHXn1RfVpyPfI5bUPfvm5Jv1588WXDvn1IN+2bduG5d6uvgW1G+53Ppwv3bLzsdvT8lwW3tSkl4bPpTqMxw69pM/itu3PPb12mUpLKa/Z8Jw+i3PdLx2eXaf8rMubx0D7Ot3ZnWXVmnXNW3abr3hUrW/v3mk+OhozK//Zw6+4tzrjMi9YvuG5LjNhbS7MpH5eOPNj9feqtbdnDjwtC2/bVr24qrq6NjDpdJ0VTfW2+xsySxSYGDzco67tbdu2hzNvSK5qnKpdS5u7nkt/W8h9drbp/o+jvUf8a/Fo/y7Z5hdLVc0vRjPfvrN/nzQ91OQ7bbViqZ4+H6haeu22YEojwLjD9+cGz6RuoTMvbPXt5kZJvlPJhk8eUX+nD/CbCc6BpZ/fkONnY5bXb1Ct2MmeI3kjjHMmgab7G7PfF4FATUNT1ZLq6hWrKybk6Y8NHtYDQvJd5nvkBQ33b9OnevToYPYTd8WG+qwqu9xpcJc1+kyTW1qjXx0YcvtvxgaP6m7Dxgcb/edtL2ho1G/ogcN5pwhMZdblnYWyJbc1bmr/at/Xsx4NMyv/mYED+gneuGG1z61ecPv9m5wvHM7WzOqn6djvecmnT2j5hte++/XXXsvqALryyhrSPuoZo5OTK6qrqxbX+15bwe8yuD/kUb1pQ1ZQFcobwo0yrK5aWjaa3Yk1keh8uKXHiS4ZnxiU2b5P6lfU+N+9VGlyvH4tKJEAE1he06DeOAOD7miaM4EsY1ilomaF+gzpmUrm/ERM+gD/6NCgftguvXjuZA7nLpqO3yFPYMvizlDLWZdqHvz6d8Qbe+eDt8t39nBCxzv/d5nmFDhwJKE+PaWpXu73bDN/r6jy+Qy9IKBz8xg20x6WlI0OmeJmGQ3ox/5Awo1LMzDr8uaT+U3+9u1N7hBCdXPXt76zb+cja2uynxszK//EuSF9q1fUV/vkKSxbEfZ+ipm5mdXP6tVN6vZPDrQ33X3f47sOHEqc8XmuFptvJQ0sW/30l1977evfTf/RsomxsbNDJw8d6Hxq8+M9Js3D+X5bztpVvnqnDKtf3LE289158VDnA5FuVd+WNG17yCe6CLN9n1QtyxXNy/RE5bJrOL6UTIAJVK/Q39MfGDLPH6fbK+t7+c5AvzuVzPmJmIxIdM5UzMDgq09syWWv/vDi/eSfbXLMfOjOXZfSTEyoeJfjXWY4Q4CT50bn9CO9Y/SMuZAXenebwmZr69aP08BwvgA7Favlzfwm/9oNO77sfHv/dHfL72zMGhHWZlb+i6OmHlQv9f+MLU6oImcUnZaZ1c8FtY/s3KTH8wMXh4/27G6LNK9bdaeONUP5J/leQfl7heQk5f27Ht+o52WvWrOm6YEtbbu7D+tJZxkmzS3JW7v89Xbr6CJc6Nn7km9Fme37ZEXNErOUpaJCdb3M4MSvHiUTYJzBFedXL52Gsc/38qsbdCw6rIdhnJ+IKZlfiBk959TqgoyN+byr5pMilHfZ2qe72urlY3by9KuRAifCTU/Oj53lFTmfKFeECDGvfetFd+6gpmPNA02rPrHxmdiZEggzuS6fmW28pX1vz9HT7rxs8TF/SfWKxk1Neih2LlU1bWlSz/nh7mdetlBPyq7lDrCplUyAcQdXzK9eOr9w6fO9fCcWmSkBpwfVlrl/IaY5emxqc/hz2u7gXmFsDwHWtx00ZcxjNr+0XpzyLgs/rUOMenTkaMZIMyx/7kfRXD+kZlA/K26/f+e+7x472vNVNcFXzaXVJk/3tj/QcsD3V5eKbiLxQmq2cdUqNUs52n2wr+/ose9957Uv73wkPK2aNDU1I/nBbU9v0iHm1XwhZu7fJ+aNsbTC+1HgGlM6AcYd6Fe/emkmkPl/L98d6JdTAs4M9cstsyPR0hrzcejCrPugnPfv8NmCOpIWmIHYweE8/W7nhvWXA8sW2PkEVLHcvFmHR2fR+1WQIpW3Yu3OnY1OiHnqhYyvAM2s/G6wPH0u1/cqptlg03wea3NQPxcsq1m74bGd+77zPRVr3K9LDe7e3z/XYXAOnO3tfFVdu7L6tp7vfv2LO+4PN9xeu7yivDxvlSikdvlr3q6mOyyo/fx2PfnPJ8TYfJ/oYpXlL97VrYQCTGB5zQpZlSYTZ846A6A5/mEXJxYNJM44A/DZkUjUHF0zDx+ZZcu4YrmeV+CdgpAh0Xn3PRs3bnz8FTkuVFWb//c7paGTejJsYHWtqeBzzXmAnTs8aPnzbNHKW756m9OKOde9O+PRMbPyL6/Vew0eyfHvDk0MJczYSRrzFMzxlDpjoqvXTOrnWEL+Ds/G++7uPGlSHDLW3L8z+pzuPU4NZZaQ0cQRPUGvets2n0nKgvuLDF5T167R2Na779u48eGnsn4swnl4LLj9ke1movyrz2T85LbF94kZ5b+WlVKAcQZXBhOJk/pNnPMfdnF+M+b08BE9Q9kvEtWu1XUq0Ls3x8+4iyb7ffJHkDZu7OzP9ymypkFP3Qn07PedCT9xMtZz8cJpQXxcEc+9FY26D+/03hyfJCf6D7ysP4+tWrXCv4izVtHQqN85p194+Yh/6UYPbf2EfGtu3DqrPpUilrdi7fbtupRZj46Zlb+iYb35WsXLB/wqzdnD+31mOrlTGM6JjzxqIc3Qkf1mkNhrBvWzvGxsQA1d9Bz2/bdvFpSZ3piS7JZxxt7K9b/slGni5IFXfS5T+YpV5oc+Xj3gG4nPHjkwIP/lucHAgtxVa0HDI6a5O/ziU+5PHUgW3yc1D37r2Jz2vs8/JRVgnMGVxP79alglz7C9mUo2eGC/mqHsH4lqHzQPn+EXH/b5Bw/PHtr9zKvD8keQhmtX53/s1YS36E/KA7t1I8VrtH/vbv39sfrPh1U/XfnqTVvU5/TJniceztp+LPHKw0/oHao2PZj+5cC5VNH4kO4YmDz81MMvnMx874jTeLx9YFK+NctWZ3/vZgpp7/RilndZeJv+OqK8zbu8D4CZlb987UO6LOJBlPn7aWMnO1v8/x1nZ/K8iEuZk5VG+3c9pb/Il2kG9bOmUX8NZ7Jn997sH2s8G3tVR7+l9TV+TQTf4H/FuE22wVf3Z0UKWUlaup1OsLRJneWrH9RjKOe6n9md2UiZOHNg917VMFrafH99vrqVau5mhBir75Nr3fVf+MIXzp8/v2xZrkmZV9SCsvcGu4+eu3D+/Adiben9v/d7v5UrwvxyIPlnf/kPgQsXLoiV6s8+svE3P6xf8PpQzceXJnqOvvVBYHL4aPf+/uGL1y+6fix5LjGw/8+++NSz3T9+V25V1fynz/2XZTeoPQKBS0N/+edH3xILtZ96qCGo00ROt3w8qHL64PyJb+8/MvwvgffL3vsXkdGr/+Ox9pd/KE+irL6ta+tvmSpe8Vv/WR9ZbX8gkfzQ9Yuc7f/wxaM/k+Uru23L80+vrXQOnOvQjmT/y9+W76RfW/3Ze2o+pNNcvrveEFxRO/bX3xal/ODC3/emzmJ48Oifv/C4exr1bV/2+wKan+tH/15edfFpb+jch5cufi95IVARLBclmFF583EL5FvcNB/+jVsm//rbfy/v5fm/e2vpBnfzGZbfLcu7p/9P9/7B0fIysdPwYM//fKb1S0flndbST6ziw2X93/i784HAuz/+9l949tnV2v7VH71bVl9/x1tvieKk7zSD+ln+G7foIr37D3/5F6JEN3xw/YTYQxZp79Y/7P5/qkSNT+/8dOob7zlv2vQ49c+/dmr5a3Cw4vrDPSdEmfQ1WhBQp64L+4cv/935D8qqq3/twgVZ6LT9bwh+/DdNofUd0Xuqy/vU7h5V5qrm5zs+ZS5SrrNQb+Ju+cq7J4YqGj/lPDJmVE/yvh2VROed97a8LI015LxiV7vrxsfHT506tXLlSpNQZEOv/M4DL+pPL2XN0e/laV2OHXlqzRP6K1Jlm6LfeyTnlqP9nVuf6NY/KJKlrLr5uecf837xeTS2Vf++eXM0s3F79sgzTzzV65vT4lXbozszfsZi4kxs9xPt+hf0MpVVr9++c3s4fYc8hxZEfY10i7/r2w5mz2XJvav4iPdUxPx0Yza/885non/XmhbdGlGqt3e/tsF8qpt2efNxC+Rb3EwTJzubt+jPv2WNzx30/mbMzMqfa6+qpq7Pl7W0+d6Hs4cef6DN50CLV7V99YvL96u751OaadZPQdRD89snPqrWt3dl/KpBvptWOKf++ddOLX8NFlI/OZlJl7Qipt//2ddpYmh/y8NmBlqGzIuU7yzECTSp1wK3be/e57kK060ned+OinvB8l2xq11JdZEJ7r+gnOsfdnG5v18mtlyRb8uKhsde6+uJtm1qXFHtzOfUs+7boj19r2W+e/NYtvrp1w52d21vWuVmtLhK5XPwu1/MfkotWB5++ut9mdtXr2ra3tV98LWnp/G0nY0Fyzd88bvmLKqcjnnnNPr8zjufBQ07vvXillR5xIdit7OjiOVNDeLKfo4XvANBMyu/2utgtK3ZKYq80dtePPj1HQ052/rL1qbv4uzz3S/6D2kb06+foh461zm7RNm/mZP3pl1hy8LPi3fQtlRRnZIe/J4qqfst6gOZ/+rAgpr7vywvr7hMTqHFvqZyFf4mTvWo/nj3M94O1bl9n8AotRYMUOKm/ugKQCu1FgwA4CpBgAEAWEGAAQBYQYABAFhBgAEAWMEsMgCAFbRgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWFPl7MD/5yU/MEgDg6sIXLQEAVtBFBgCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsKK4AWY0tvXOO+/sTJhVTSVujY2a1ZIjz8/K6cmMM66Fkuh0j2ft2PaJUviWLj9P2S3zv7RX7vjz+d7OyDVV3pnV/uldoyk3voKV2VXcAFMRfj7aHOiOeC994pX2gebo8+EKs46rQ+1jx449VmtWAExNPCCPze9HYdG7yGofSwsxic5Id3NUP4hkRDbcCJQepXN9LpDprlTWd3bGdJY6B0/+uQK7b0YOuX9nwt1EvJzK0bOtXzGyjy0S1rUPiCuRcS4i+0h3YKB9XWrnoVecfb2bek7V3XImRfYh93I3T1tRR9BHmypreX5qU3dB8uaWVQKRkFl2Te7lFEzwHi/XaeRId4+59ZUhk5SSffzUKfqWUUgdR52g3kzslnYfUtkIaWVL3duMIhueYvifg7o0aZXS4X9Q3+uSlSgT3JfTVlTZdGapA6S/mlYD3Y38LrjkOU3vJfDLPEW+mtrae4KeorgbpG2dXhoPz55p5UkxecgNc9RGX36n5E31VJzUyYkDb43F3KN7dvSVfZ4iJWdlTmUn0tLv16yNj4+fOHHiF0UV31NXV7cnrhfk38I7rz/qtywXH339HZWctn2KTHQ3cbNOWxTSMvLNJ1dG7p7qtMx+etmzYnZU6Sbr1LL3eKll+bqT6iW2cE5DH8Zs5Dk/b4aeo8vkVJaeF9L3mJLnzNLOQOaiFgvI2k1Ne9nd0ZuDZ1ls7OabknYSMj+zTa7TyJGesafnbru8x/dsnrbskSNHmeyWOW1Xzytpp6BWnD0caYlqJfsUcuSS46ByycnQXfZLlDmZNH0EzwbuTs4BPMty0d3W+0LaebpkqpPmWc6RuYdMdQ6TOld1EJPsWfZunHZID+82znLaoeWK3kLl7Gyctk2Km12eUzK76ez0Surk1OG8W5g8PNyN085B7egeJC3ZySJ1FO/Wc6MkBvlrHzvYVt/90tatqdaL7igzXSoV4afl6wVGVNkX4zYra9c26wWtea3ppBH5B9qedrcSzajs/PNl5Khve1DlWNHQWJ+2MjB0TiwVUIxpdx05R1GnpA4zGnvJvXD6MIH2V5zPJNMqcg6yQN2HVI7nhgaam5vNSuJQt8p+Fln7KahjIPsy5DyNHOni7APND+lkdW/Uy7nIi1zv5iIzGUhdZEPk6G5SEX7Iv+rlq1bpe5ur7JLXxa0rqsb588mlgLrsWxGdxClqQN6Lk6qB07rgbi0o5MrLIplTGu0/PKAPWcC7Lwd9SFPB3IuQ5yL61EZfOU4pX8VJca+dvPXDZ3KWZOqbXdgTYy6UyCwyVcSB1KUfPTMcqK9ZqpaliuVVZqlQTosz0m0S0sj8ZWvR5b+ZlD+jNFXLzU115CpG7YOiYkVUtlO1dQsh3vKyb80l+9qyTKPI2eSJqxqtHigP1tSnVsQNm1XWhnxbmTxm2jbPdRo50jNuzlRVTF5k7w1eWpP1fJQ5ejbx2SLFv1pl7O3/EDFdG753WcmZS9ZBfSuib2L+GlDAxSnogvvWgoIyl89RXVAZANUjMuN4U95hj8xDek3jgZAlxylNp+IULN95yvJN+cSYEyUSYPSl9l76mdMXdl17VfSYEM3xaSAgPnQclBu43JjvKDSjGVCfRVWe6jbPQXdnszrJFOfTSZopiuzplM08I/H+HTjcPyreCeImiU9QAb2S+rAz5dWckvzcdeyYaMvqJ8zMIm+u05j96c2ZWVQrfYMiw6ow4kqZ5ALkOqhvRfRNnLIGzI0Z1wJzfm7Ms8DiA2FOFXaeBT0xZq1kAkw6GW68DU0Z4gukWsjy2uW7YJn5+ykoo/ymKoZ6N8k3k3xnzFzuD7seBRRZv7mVrAewfP8O9fcfDjQ2VMi81EqVficXcjULpR9u8tmW0T80tVynkSM9I3mqKpZ5keWHwAwyR88mPlsIeatVxt4ZH6LFs1MFyqmiY3YuU9Vl34qYkZivBhRwcaZ1wdNqQUGZ6/M73B9LxZfM+z6Nh4jvO8reA6GgilOwAs6zoCfGnCjRAGOa6ebzy2jsmfYB3QGp7pB5F8iORPV6FufaiVAe8d8kLX/90dDvw9LUGeWXoxjew8nqUC/fszOnOxaecRod8gOMT5uowCLnIupkd3v7gH7m6RXzdJlm1rJyO9FDdkmrhfSTFk/SmfQE5zqNHOkq2XTKq3ujXs0l/SLL2Y6pTnqHfMY5m+SunXmqlTu44J+/2MB5OiU6c3druLmkDSX4HdS3IvomCnlqQCEXp5AL7lsLCsvcXP12T/sl7b57HiL+NdBLH9IZlfCclaUHQqEVp2BTnGdhT4w5UQqzyBTvFAdNT6dQvPMa1EQH6dHX42IL/9kUxp54asaFd96E4mYk+E6c8M9ILqgzdRecbd1MslaM1FF8j222TG1l6I3lodIOmVYiz1FSW0y/yHmoI3gPlzoRaaqsPeeSOtc9cZFs8vEtgck2/VAZR/fknPs0cqS7yXtezyqRkn78VC4+myqesskc9aHkbqmDegrqrZ9ySe3jvKS3TpN2fLmSdRrq0uzZk5VLjoN6cvRs7Jto8nDW1YHSjp52cp40Tw6ejXJccM95pr3ql3mWrKNllNukeVO9NTCd90zMrmmZqRX5glzIU2LFm5qWi04SUqmeipPKOi3b9CM6UqlpR1AreleZiXsBPRulMvMcxv8g03WdCDCnTp1auXKlCTiAFeJz8Us1B4s49nHFic+F64YemkWHykzIgx5uvKauc4qoY4fWXuELbkFRKo4lpdpFhqvMNDrA5y9v55KekTr9fj7MmMXxfcuu3opDgIF1sot3ndMBfjVTX7FxZn9GAtGr40PofCCrmLzi8/SCX70Vhy4yAIAVtGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhQ9wIz0PbstGjcrRjza0pKZBgCYX2jBAACsuG58fPzUqVMrV640CVeaaMHsGr53byRk1iXRgokGIl0ybaSvoyOWVKnBcGvrmkq1qPZykte3PqmTxW7Hq8LJmHghuP6/1Z340+E7wiO9erNgeIfZyruvJz0e3fZ61ea64/vUS/JYlTHTtPLsKw6xzzStQpvVCQIAchAB5sSJE78omp8d/uOWl39kVowfvbx1q0pzF9KW5S5/fPhnKlUktzibiC3cZbHRs1u3OjnLfZ9Ve8h907fR6SqftGXnEKnjyvxTu3rOAQCQbf50kYUiXbrJEI/FAusjTlsmFNkcisf6RvRaIFTnaVUE14f1mkxNDsumSeWaJ/e6LY/K0B1BvaSFwjrXUF1tIFB7r7Mi9h0ZEQ2fWDy02d11TWR9IBZjoAgAcintABMKh4OJaMu2lpbURICRkWQg2duhEtUfp89qOuJRtW9Hr9NVliVYlRZ7AipAxfc5B827LwCg5Af5VYOjS47QqJDQYVoqwfWtXTLd+eOOzUxNh5boiMqhdX1GFJlCaLPnoOIPgzAAkFPRA0xlZaVolDg9XIpso6Q3IEIR8TTfEQ4mj8dHKiuDprtrJuLH48Fw694uZ9S+cPKMMk4UAJBb8VswobraZGyfO4iixliSwbqQDADeL8SMxI/rZN1v5naZTftLM250iken081VuSYcSvZGnRMd6etoedZz1gCAdCXQRRaKdLXecbzDGdto2ReI7DUNjFAkEjJjMC0dvZUR3RVWuebJzSF3OMSZ0FwYkWOtGYBpiVW1bg4FZKvIvDiFUKR1fSC2Sx+34/gdzvRoAICPon8PBgBwdSrxQX4AwHxFgAEAWEGAAQBYQYABAFhBgAEAWEGAAQBYQYABAFhBgAEAWFGcAHPdPGROHQBQGFowAAArCDAAACsIMAAAKwgwAAArCDAAACsIMDMzGtt6Z5rOhHnFlkSnc5DUEgCUMALM9MkH/LrDjQePpRxsG45YjjK1jx079litWQGA0keAmSYRXSKB6LFjz4crTIpUEX5eBJn67sjW2KhJUnFI2RqLOW0O2fDpTLiviLRUUygVndKaR06OU7db3Gxz5OUmig23xhLOC/IAnnN1zx8AZml8fPzEiRO/uLLMseeVuDzx+J66R19/R5XhndcfrVNEgklWSXvUhp5FuZOgVvROnk28Kzpn7556RafLXHRyaskjlYFn2ZuXZ1mdkNlYn5xnJStnAJgZWjDTkTjU3fyQbLqIdsG69irRkBEeGloX6Q5ULa8IVDQ01geGz4g2wGj/4YHmqOnRqn2wrV4taPVtD6p0tbV3ZWDonFwSjSG3L0xtU4jEK+0D+tQEmYVsYalE5ywqwk+LJtZLTgvF2bh2bXP6ijp/AJg9Asw0yPiyVjyuR2PPtAfaDjrxQz2iZXqgYnmVSgqcGxqor1mqlz3J2WRc8mU6rda1D5iEvEbPDAc8B9QyE3OeRtaeADAHCDAzIMNHY4M3MphHtHyk5wwZhdKhJTLcJmcRHExr/ADAPEKAKZTptBJkGPEQ7RoTblSXlGrKLK0xHV5Kxg75yexkbEmfRTAF2TjxHFDLTJzWaQDAbBFgpkGEDTlCocZLDveroYrR2FYzAKOWnBEPuUl3xEzbknFHLRTKjQqJzgK7yNQ4T2qARTaCxMFVonMWsl8vNUwDANYRYKZBhI2ADCwV4eejVe3rZE/WM4GH2uoD3RHzzRhncF5t0iyThZdq2poLH+eofczdUex5MNoccIJZFjkF2ZlX7Dkn3b8mT0Ukmm/oyNGcqihfpAFwBV03Pj5+6tSplStXmoQrYj7+8yq/ULOrRdtAfg0mx4PamVyW+bLYS8SKaXV6AcB8Rwtmemofy/Wlfdkt5bZivG0LsfKSM0oDANcOWjCF0i0Yh2zJdJtlrb4trYmiGjNm/CTjJQC4JhBgCpUeYAAAU6CLDABgRXECjPyRmvnGnDoAoDC0YAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVlw3Pj5+6tSplStXmoQrbaTv2V2xpFlx1Ea6IiGznCke3RYNbBavj/R1dBy/o/XJNZXmFQBACSmJFkwosrery/snZ3QBAMwXpdtFJhooLc/2jZi1eLRlWzRuVvKQe0X7+p7d1tIi/6QykE0lk+hJF9l29MX7ntXpMlUeSG3Tkdo1lbitpZCTAABcnWMw8d7he1VLKFKbjO1TcUJ1xFVu1i2k1nDQSReSsdcDEZm+OZTs7WiJVbXKbSKhZCymY4mILvsCpo21IzyyzxO0AAA5lUSAiUed9oH84206zEhwfVh3sYXqQoHksBzgqVzzpIg3puOtMnRHUC8poXv1KI7cOhAKOyu1gZERcSIjfbF4aLO765rI+oATeQAAeZTgGEyrvVF7E8k6ejNnFTiCVd7QI8kAFd+Xin+59wUAeF2NXWR+dGiJjqxvFTGsdX1mGMkvZPrWnD/MQQCAqV0jASZ+PB4Mt+7tmv6cZtmmUX1lAIBpKd0AU1kZDCRPxNWjXQ6EqMRZ0IMxQjw6nW6uyjXhULI36owLpc9tAwDkVMItmFBETuXqUF1bgXB4er1aGURetc5UglhV6+ZQIHlcx66phSKt6wOxXXoMhq92AkCBiv5NfgDA1elaGeQHAFxhBBgAgBUEGACAFQQYAIAVBBgAgBUEGACAFQQYAIAVBBgAgBUEGACAFcX/Jv/IyMj58+cvX75s1gFcqxYuXBhUzDrmuSIHGBFdxsbGPvKRjyxatMgkAbhWXbp06e23377xxhuJMVeHIneRibYL0QWAJh4F4oGQTPKv+l0lihxgLl++THQB4BIPBDrMrxoM8gMArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsKHKAWbhw4aVLl8wKgGvee++9t2jRIrOCea7IASYYDL711lvEGACCiC5nz569+eabzTrmuevGx8dPnTq1cuVKk3DFJZXLly+bdQDXKtF2EdFFfO4065jnih9gAABXJQb5AQBWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWXDc+Pn7q1KmVK1eahCttpO/ZXbGkWdGC4R1Prqk0KyUmHt0WDWzuioTMOgAgh5JowYQie7u63D+bK2O7nu0bMa8BAOankmjBDN+719skGOnr6BgOq1ZCWvvGbdnIDUxqMNzaqls7WYly3+N1nl2O39FqWkbxaMvxOnMAd6/aiGmYyFerwsmYSA+ul7uInDp61Ua1oVAiTgsGAApQymMwKrpUbtYtm9ZwMBnbJxs28WhHLBjRzZ1IMNYRjYttfRIrQ3XB5LAOHiPx48mAsxKIH4+H6pzoYvZqDSejLSorKRETEU6kO9GlUrexIoG4swUAIL/SCzAjfdFYUj3/K9c8KZ7ppq1QGbojqJe8QpGu7NaEkyh3EbFEJonYIhsfeiV+PKHyj4s2Sihidq9cszkcTLzudM2pDaSR+IlkcH1Yr7lbAwCmUhIBJh7d1tLi/OnoDYR3eJ/j5lXTSSWe8mERCaJqY7fF4Z9YWVkZSI7IRo9ssoSrgiNmpVa1X8RKsCoVteTW2WSzx/OKdwcAQB4lOMi/1wyUOKElOrK+VaS3rnce7apl0yWHbdQGHarV4Zso2iHJ43ERSZIiMIgGTUCtjLjNEwCANaU8BiOaGsFwqxoIMSlpVFjaEQ7KEGKSshJDdbXJ4Xj8eKAuVCkbInLlRKWOL6p94wzLCKp5k0U2WTyveHcAAORRygFGSI3LR90usni0xR2Nl4P3QRk8fBMFER/ivbFkUHVyiZVELCZekq+I6BMOB+NRs9dI375YsvberFhWuSYcSvbG9Ebu1gCAqZTiNOUUETaiCbUkmjLh4Y59I2qmcjyaGmgRTRa9r2+izl9ORVPr6ZOVpRzTlPcFUjmoPHR4C64PV/bGmKYMAAUoeoABAFydSryLDAAwXxFgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYQYAAAVhBgAABWEGAAAFYU/5v8IyMj58+fv3z5slkHcK1auHBhUDHrmOeKHGBEdBkbG/vIRz6yaNEikwTgWnXp0qW33377xhtvJMZcHYrcRSbaLkQXAJp4FIgHQjLJP4pxlShygLl8+TLRBYBLPBDoML9qMMgPALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsKLIAWbhwoWXLl0yKwCuee+9996iRYvMCua5IgeYYDD41ltvEWMACCK6nD179uabbzbrmOeuGx8fP3Xq1MqVK03CFZdULl++bNYBXKtE20VEF/G506xjnit+gAEAXJUY5AcAWEGAAQBYQYABAFhBgAEAWEGAAQBYQYABAFhBgAEAWEGAAQBYQYABAFhBgAEAWEGAAQBYQYABAFhR5B+7nJiYGBwcvHDhwgcffGCSHDfccMNNN920YsWKBQsWmCQAwPxR5ADT399/3XXXVVdXm/V0Q0ND4v8NDQ16FQAwjxS5i2x0dPSWW275txxuvfXWd955x2yaQzy6raXF+fNs34hJVunRuFm5Jo30dXgvCABcYUUOMJcuXZrIK/8/RCaiSDSwuatrr/4TqeztuMajCgCUjCJ3kX3ta1+rr683K34GBgY+85nPmJVM8WjLvkBkbyRk1r0pKvbUhuIJHW9Cqc1G+p7dFUuqxdDmLpkqU47X7XhyTaV6uaPj+B2tekVmeLxObeQQKbGqyB3Ho70yj+B6Z0v1SjShFwOB2ojZS56SiXnmcEJWotw34N3FLZc4HxFE5UFS+QfDra36qOrVOypjvSI3VUY352BI5DriFgQArrR5PYssWBUMxF/39gKFIqId40aDeLKqVbZsWsPBeFS3bFR0qdSNnh3hkX2qwVMZqgsmh3XMGYkfTwaclUD8eDxU5+bnSMaiw/e6bSbdDSWf/smwOlxX1+ZQIKEOKA63byS8wz2c2tYvUR4lOaJLIg4qNjIr4nwCdSEdXZz8I8FYR4db7GTshC6mji4m59Y7RuKmEABQFPM6wFSueXJzKNnb4YzBZPaOhcL603tl6I6gfnzHY7FkbcSEoMo1kfXBeEw8qeUG6rEuHtfDydpQKKFX4scTPvFFxjGTRSi8Ppg8ERc5i9DWZVoVYsVvJ3G2e7OaE26i2CV5XGYkQksyGKoNmBURX+4IVY70vZ4IhjebveXhk7GYU9qg2EAtyCLU3mvKvCYS5h82B1BM8/17MLLJoj6wrxdPUzXgn28MRjy7VbPHUVlpnvdyQUcg2WQJVwVVA0Ks1PqFCk8WakenvSMO0PdsWqirXHNvbTK2K30Cgm+izFRlpJos4bpKs3JCtV/kinuygrcQrozSVVb6bAMAV8x8DzAO0RLQPU7BRFqfWaFMA0K2HqqCokEjGxCCb1PElw4tHb2VEd1bZZJVy0bFP93S0pHHL9FpRYlYEqysFAeWrajksIovADAPFTnA3HDDDWYph+uvv94sZYtHs9orGQ2KTPJDvfdlZ6BDCNXVJofjZsBDZCNXTlT6xxdPFjIH2WoQTY1kSA7tpEKLl45/kVo9vmJkJKqRoJE+M+ojMh0RKyO6+0sdInWy/mXMKJ1s0ABA8RQ5wNx0001vvPGGWckiXhIbmJVsobBor0Q9vUyiGfF6PBgO52516F1MVBrpi/YmnXEa+QyP98Zk60GvJGKxYK72izuzIB7z5OBEADmzy4QR0a7Z5p5g/HhCjc74JgoyrMViCRlMVLRIxnpHTPtF96rtc44qSuBXTFU6c2qicGaqHAAUR5GnKf/sZz/70Y9+9M///M//9m//ZpIcou3y4Q9/+OMf/7h37CGbnI6cahS4k4NVesCZFiwe6R0n6szUXrGSMU1ZU+mVJiV9srKXnKkVCNXG42rGcGqaclq2rVWxjlhQnow8tJrQLLgb+yYKZqqYPk91IO8kbPmqzzTl9POUe6nrURsOJ2P+RQCAK6HIAWZyclKcwPvvv2/W04kYs3jx4rKyMrNeErKe+wAAP0XuIhPB46abbro5hyVLlpRYdAEAFOpqmUUGACgxRe4iAwBcrWjBAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsIMAAAKwgwAAArLhufHz81KlTK1euNAlX1oWfX37hfydODiX/eWzCJKFU3VS+4I7/EHz4k7VLblxokgAgtyK3YL7UE+87+VOiy7wgbtOhH/xU3DKzDgB5FTnAHPu/PzNLmCe4ZQAKVOQAM375fbOEeYJbBqBADPIDAKwgwAAArCDAAACsIMAAAKwgwAAArCDAAACsmC8BZsWrXU1HzJ81O0xi6fnMGnGGsd+/xaza8bu/v945hL4sJXxBAFzD5kWAuaWro6rKLAvl91zDj1QRXf7g1jKzEhjc1NKzuqVvl1kFgBJS5N8iW93SY5byWH1XrOnm8kvnv9T6t9+QweZjv70oMHy8Z9PXxGviI3wq9ujEHa1N91RO/rCnt+WITFSrftuPDK/uGNSLHiZ/beyNH4X/5CdiwWTyxvklt95crl75Tuqx7uY59p3jgXvqyt29vDyBIXVukmj01KksPYdzi/zDn9/825Wpl9Kii7wg731SHlqdTI5dxILPFTAX01mVC7kuiA/RSjNLAJDbfGjBHHnvgvj/opv/q+wX+klLq/jMrp+VGS2bQFWdbNnsGh4LBMpuDZlOpP8kn7Zjp7KiUaCyKrsva0drKroI5bf+x67VZlmoMtFFKL+ndYVa8J5DuYguZjFderOj7Leb1ptsPdFFKL/1Y0dMtsoiEyqEjDPJaXq7TH1BAGDG5kUX2eCpEfmXfP7KIQfn6ewEG/XnRz+8JFIW/Kp46WsXhsXGH1n6uyLhM0vkA3TkgmxtqGXxud7dvvzWj2Z0te3qcDPs+Y48aNmSj+hXFPEZX7x0XASwQODGD6n8P6oCkmhDuLtkW/FJGV1Ew0Vu86U3Jp34d0tXSEYX0bZQRxwWpx2oXOqJCnoXXTR5Jt/4k161uyqFaYJkyNwlnwIuCADM2PwY5BfPff1gVUQLoOnVz5gVYUeriDrelocKSIt++a7VgR1V4gk++cPvy56f312yQPzfiVJ6exWQsogGh9jmHqcp4BoeVj1IKoBpOs+xN97U3WWq8ZRl9YeWiP9fevdvVbeYCBLigS47r1YvvVWcw6Xz35atK2Hw2yr2pKKC2eUnf/u2W/apTGeXwi8IAMzAfJlFZp7LzmfzQFXVCtU9JRs0v/r9VLrm9JLdJfvHnCe7n8zP+CpWNX3ygjxWjubI3NFdf1eYjnY5TdXoAYCCzYcAo+b+OoMTP2mJO60E0wLQ8eOXl6RaME4v2a03yy6gt8/prqRvXJD/6ozTI2T+qLEcl3fA5pZfvVGl5aXzdHqWTJdXJmcM6ZO61ZUqzrsXRFB0052etAtv69U5tmSJHF/53dAvu6dYwAUBgJmbDwFGd0lVVqmenCY9Km56q4RFN/+BTE8b7Xe6m4TJN+LOhC4TdXSPkPrTcZccR8lUfo98NW20P6evvalaTvl3MSdTVacOKs9f99qZYGnSdRFGzqUmmOWmSlHoXO03xuTRdcE9cw0KvyAAMBPzY5B/kx4Ad5gZt0f+9ivuwMzIsBp7dyePBb4Rf1c+vNP6xzLyGftO5jj54CY9gC9cOv+lnvNiRfXF5fGTllY3z8kfvuHsns4dnNeGjzvTlL/WZ6YMKLIxMdVE4W/8yTnvpSjEN/7kH9z+w+Hj3isw5QUBgJmbD9+DQYkRbR2zBAC5zZtBfgDA/EKAAQBYQYABAFhBgAEAWEGAAQBYQYABAFhR5ADz7xfeYJYwT9xULn/BDACmVOQAc+dv/qpZwjxxe/XNZgkA8ipygPmDptDalb/Oh+J5QdymNbf/+iO/EzLrAJBXkb/JDwC4WjHIDwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwggADALCCAAMAsIIAAwCwQgaYG2644V//9V/1OgAAc0IGmMWLF1++fFmvAwAwJ2SA+ZVf+ZU333xTrwMAMCeuGx8fF39NTEz80z/900c/+tGFCxeWlZXp1wAAmKFA4P8D3gzoNJgUaQoAAAAASUVORK5CYII=
''')

{'id': 'chatcmpl-ADDlm2UQzltkcrBuLW2TarXYuSfOd',
 'object': 'chat.completion',
 'created': 1727714558,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'The image displays a login or registration page for a service called "Request Tracker." It prompts the user to enter their username (partially obscured), full name, and password. There is also an option to show the password, and a button labeled "Save and continue."',
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 14180,
  'completion_tokens': 55,
  'total_tokens': 14235,
  'completion_tokens_details': {'reasoning_tokens': 0}},
 'system_fingerprint': 'fp_74ba47b4ac'}